# NA treatment

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import warnings

In [2]:
train = pd.read_csv('../data/train.csv',header=0)
test = pd.read_csv('../data/test.csv',header=0)

/var/folders/09/t9n5xk9x22j8tr_9772wtssc0000gn/T/ipykernel_16580/1621859618.py:1: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../data/train.csv',header=0)


In [3]:
# Transform the time in minutes starting a midnight
train['time'] = pd.to_datetime(train['time'], format='%H:%M:%S')
test['time'] = pd.to_datetime(test['time'], format='%H:%M:%S')

train['hour'] = train.time.apply(lambda x : x.hour)
test['hour'] = test.time.apply(lambda x : x.hour)

train['time_min'] = train['time'].dt.hour * 60 + train['time'].dt.minute
test['time_min'] = test['time'].dt.hour * 60 + test['time'].dt.minute


In [5]:
def actual_time(df, col_var):
    df[['hours', 'minutes']] = df[col_var].str.extract('(-?\d+):(\d+)')
    df['time_diff'] = df['hours'].astype(int) * 60 - df['minutes'].astype(int)
    
    # Calculate actual time in minutes for each bg measurement
    df['actual_time'] = df['time_min'] + df['time_diff']
    
    df = df.drop(columns=['hours', 'minutes', 'time_diff'])
    
    return df

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/var/folders/09/t9n5xk9x22j8tr_9772wtssc0000gn/T/ipykernel_16580/986835876.py:2: SyntaxWarning: invalid escape sequence '\d'
  df[['hours', 'minutes']] = df[col_var].str.extract('(-?\d+):(\d+)')


## Blood glucose

In [20]:
bg_columns = [col for col in train.columns if col.startswith('bg')]
bg_df = pd.melt(train, id_vars=['id', 'time_min'], value_vars=bg_columns, var_name='bg_time', value_name='bg_value')
bg_df = actual_time(bg_df, 'bg_time')

bg_df.head()

,id,time_min,bg_time,bg_value,actual_time
0,p01_0,370,bg-5:55,NaN,15
1,p01_1,385,bg-5:55,NaN,30
2,p01_2,400,bg-5:55,NaN,45
3,p01_3,415,bg-5:55,NaN,60
4,p01_4,430,bg-5:55,NaN,75


In [21]:
def fill_na_values(group):
    # For the remaining NA values, use the mean of adjacent non-NA values
    group['bg_value'] = group['bg_value'].interpolate(method='linear')
    
    # # Backward fill the first NA values
    group['bg_value'] = group['bg_value'].bfill()
    # Forward fill the last NA values
    group['bg_value'] = group['bg_value'].ffill()
    
    return group

In [23]:
nan_count = bg_df['bg_value'].isna().sum()
print(f"\nNumber of NaN values: {nan_count}")

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    bg_df = bg_df.groupby('id').apply(fill_na_values).reset_index(drop=True)

# Check if there are any remaining NaN values
nan_count = bg_df['bg_value'].isna().sum()
print(f"\nNumber of remaining NaN values: {nan_count}")


Number of NaN values: 1371684

Number of remaining NaN values: 0
